<a href="https://colab.research.google.com/github/cooolbabu/GL-MIT-ADSP-2023/blob/master/FinancialAnalysis/Stock_Presentation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install yfinance

In [1]:
import json

# For DataFrame
import pandas as pd
import numpy as np

In [13]:
import yfinance as yf
import json

SYMBOLS = ['INTU','CDNS','WDAY','ROP','TEAM','ADSK','DDOG','ANSS','ZM','PTC',\
           'BSY','GRAB','SSNC','APP','AZPN','MANH','ZI','NICE']

for symbol in SYMBOLS:
    data = yf.Ticker(symbol)
    file_name = f'{symbol}.json'
    # Use a context manager to open the file and write the JSON data to it
    with open(file_name, 'w') as file:
        json.dump(data.info, file)
    print('saved to {}'.format(file_name))

saved to INTU.json
saved to CDNS.json
saved to WDAY.json
saved to ROP.json
saved to TEAM.json
saved to ADSK.json
saved to DDOG.json
saved to ANSS.json
saved to ZM.json
saved to PTC.json
saved to BSY.json
saved to GRAB.json
saved to SSNC.json
saved to APP.json
saved to AZPN.json
saved to MANH.json
saved to ZI.json
saved to NICE.json


In [14]:
# List of stock symbols we need to run fundamental analysis on - any symbol added here must have the json file
# containing stock info from YF
SYMBOLS = ['INTU','CDNS','WDAY','ROP','TEAM','ADSK','DDOG','ANSS','ZM','PTC',\
           'BSY','GRAB','SSNC','APP','AZPN','MANH','ZI','NICE']

# Path to read stock data from YF
DATA_PATH = 'path to accesss json files'

# Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY' : [],
    'PB': [],
    'ROE' : [],
    'P/S (trail)': [],
    'DPR' : [],
    'DY' : [],
    'CR' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }

In [15]:
def load_data(json_data):
    data['Symbol'].append(json_data['symbol'])
    data['Name'].append(json_data['longName'])
    data['Industry'].append(json_data['industry'])
    data['Price'].append(json_data['currentPrice'])

    # Could be that some indicators are not available; use NaN if this is the case

    if 'forwardEps' in json_data:
        data['EPS (fwd)'].append(json_data['forwardEps'])
    else:
        data['EPS (fwd)'].append(np.nan)

    if 'forwardPE' in json_data:
        data['P/E (fwd)'].append(json_data['forwardPE'])
    else:
        data['P/E (fwd)'].append(np.nan)

    if 'pegRatio' in json_data:
        data['PEG'].append(json_data['pegRatio'])
    else:
        data['PEG'].append(np.nan)

    if ('freeCashflow' in json_data) and ('marketCap' in json_data):
        fcfy = (json_data['freeCashflow']/json_data['marketCap']) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)

    if 'priceToBook' in json_data:
        data['PB'].append(json_data['priceToBook'])
    else:
        data['PB'].append(np.nan)

    if 'returnOnEquity' in json_data:
        data['ROE'].append(json_data['returnOnEquity'])
    else:
        data['ROE'].append(np.nan)

    if 'priceToSalesTrailing12Months' in json_data:
        data['P/S (trail)'].append(json_data['priceToSalesTrailing12Months'])
    else:
        data['P/S (trail)'].append(np.nan)

    data['DPR'].append(json_data['payoutRatio'] * 100)

    if 'dividendYield' in json_data:
        data['DY'].append(json_data['dividendYield'])
    else:
        data['DY'].append(0.0)

    if 'beta' in json_data:
        data['Beta'].append(json_data['beta'])
    else:
        data['Beta'].append(np.nan)

    if 'currentRatio' in json_data:
        data['CR'].append(json_data['currentRatio'])
    else:
        data['CR'].append(np.nan)

    if 'fiftyTwoWeekLow' in json_data:
        data['52w Low'].append(json_data['fiftyTwoWeekLow'])
    else:
        data['52w Low'].append(np.nan)

    if 'fiftyTwoWeekHigh' in json_data:
        data['52w High'].append(json_data['fiftyTwoWeekHigh'])
    else:
        data['52w High'].append(np.nan)

In [16]:
for symbol in SYMBOLS:
    # Specify the full path to load JSON data
    file_name = f'{symbol}.json'
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Use json.load() to parse the JSON data from the file
            load_data(json.load(file))
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [17]:
# Create a DF using the dictionary
df = pd.DataFrame(data)

# Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]

# Remove any stocks with NaN values
df=df.dropna()

# Reset index after dropping rows with NaN values
df.reset_index(drop=True, inplace=True)

# Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High
4,TEAM,Atlassian Corporation,Software - Application,3.05,74.13115,NaN,1.9,75.3918,-0.80447,15.74809,0.0,0.0,1.434,0.715,226.1,121.865,248.0


In [18]:
def make_pretty(styler):
    # Column formatting
    styler.format({'EPS (fwd)': '${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}',
                   'FCFY': '{:.2f}%', 'PB' : '{:.2f}', 'ROE' : '{:.2f}', 'P/S (trail)': '{:.2f}',
                   'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52w Low': '${:.2f}',
                   'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    # Set the bar visualization
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['PEG'], cmap='Greens')
    styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['PB'], cmap='Greens')
    styler.background_gradient(subset=['ROE'], cmap='Greens')
    styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
    styler.background_gradient(subset=['DPR'], cmap='Greens')
    styler.background_gradient(subset=['DY'], cmap='Greens')
    styler.background_gradient(subset=['CR'], cmap='Greens')

    # No index
    styler.hide(axis='index')

    # Tooltips
    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler

In [19]:
def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()

    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    # Get position based on the column name
    pos = df.columns.to_list().index(col_name)

    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)

In [20]:
# Initialize tool tip data - each column is set to '' for each row
tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

# Gather tool tip data for indicators
populate_tt(df, tt_data, 'EPS (fwd)')
populate_tt(df, tt_data, 'P/E (fwd)')
populate_tt(df, tt_data, 'PEG')
populate_tt(df, tt_data, 'FCFY')
populate_tt(df, tt_data, 'PB')
populate_tt(df, tt_data, 'ROE')
populate_tt(df, tt_data, 'P/S (trail)')
populate_tt(df, tt_data, 'DPR')
populate_tt(df, tt_data, 'DY')
populate_tt(df, tt_data, 'CR')

# Create a tool tip DF
ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

# Add table caption and styles to DF
df.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
INTU,Intuit Inc.,Software - Application,$18.81,31.31,2.43,2.60%,9.71,0.16,11.18,35.49%,0.01%,1.25,1.23,$589.02,$378.22,$631.07,83.37%
CDNS,"Cadence Design Systems, Inc.",Software - Application,$5.89,42.90,2.79,1.43%,22.08,0.33,17.53,0.00%,0.00%,1.44,1.06,$252.71,$161.34,$279.33,77.44%
WDAY,"Workday, Inc.",Software - Application,$6.64,40.03,1.62,2.43%,10.55,0.01,10.01,0.00%,0.00%,2.13,1.36,$265.80,$158.74,$279.83,88.41%
ROP,"Roper Technologies, Inc.",Software - Application,$18.41,28.41,2.99,5.01%,3.28,0.08,9.32,23.64%,0.01%,0.54,1.00,$523.07,$416.78,$551.91,78.66%
ADSK,"Autodesk, Inc.",Software - Application,$8.16,28.13,2.46,4.00%,33.14,0.77,9.18,0.00%,0.00%,0.75,1.47,$229.52,$188.38,$245.88,71.55%
DDOG,"Datadog, Inc.",Software - Application,$1.82,63.73,2.22,1.38%,21.08,-0.02,18.98,0.00%,0.00%,3.35,1.17,$115.99,$62.60,$124.69,85.99%
ANSS,"ANSYS, Inc.",Software - Application,$9.55,36.03,3.92,1.90%,5.97,0.10,13.85,0.00%,0.00%,2.43,1.19,$344.08,$241.40,$364.31,83.54%
ZM,"Zoom Video Communications, Inc.",Software - Application,$4.71,14.22,9.03,7.96%,2.75,0.04,4.53,0.00%,0.00%,4.27,-0.09,$66.96,$58.87,$85.13,30.81%
PTC,PTC Inc.,Software - Application,$6.10,27.19,2.59,6.11%,7.36,0.10,9.45,0.00%,0.00%,0.76,1.16,$165.85,$115.45,$176.25,82.89%
BSY,"Bentley Systems, Incorporated",Software - Application,$1.00,46.73,66.69,2.40%,21.16,0.29,11.48,32.73%,0.00%,0.55,1.13,$46.73,$35.64,$55.37,56.21%
